<a href="https://colab.research.google.com/github/thisisreallife/d2l_notes/blob/main/chapter_linear-regression/3.3synthetic-regression-data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The following additional libraries are needed to run this
notebook. Note that running on Colab is experimental, please report a Github
issue if you have any problem.

In [ ]:
!pip install d2l==1.0.0-alpha0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 92 kB 205 kB/s 


# Synthetic Regression Data
:label:`sec_synthetic-regression-data`


Machine learning is all about extracting information from data.
So you might wonder, what could we possibly learn from synthetic data?
While we might not care intrinsically about the patterns 
that we ourselves baked into an artificial data generating model,
such datasets are nevertheless useful for didactic purposes,
helping us to evaluate the properties of our learning 
algorithms and to confirm that our implementations work as expected.
For example, if we create data for which the correct parameters are known *a priori*,
then we can verify that our model can in fact recover them.


In [ ]:
%matplotlib inline
import random
import torch
from d2l import torch as d2l

## Generating the Dataset

For this example, we will work low-dimensional
for succinctness.
The following code snippet generates 1000 examples
with 2-dimensional features drawn 
from a standard normal distribution.
The resulting design matrix $\mathbf{X}$
belongs to $\mathbb{R}^{1000 \times 2}$. 
We generate each label by applying 
a *ground truth* linear function, 
corrupted them via additive noise $\epsilon$, 
drawn independently and identically for each example:

(**$$\mathbf{y}= \mathbf{X} \mathbf{w} + b + \mathbf\epsilon.$$**)

For convenience we assume that $\epsilon$ is drawn 
from a normal distribution with mean $\mu= 0$ 
and standard deviation $\sigma = 0.01$.
Note that for object-oriented design
we add the code to the `__init__` method of a subclass of `d2l.DataModule` (introduced in :numref:`oo-design-data`). 
It's good practice to allow setting any additional hyperparameters. 
We accomplish this with `save_hyperparameters()`. 
The `batch_size` will be determined later on.


In [ ]:
import numpy as np
A = torch.randn(2,3)
B = np.random.randn(3,1)

In [ ]:
A @ B # this works, which means python @ operator support tensor and numpy.array
# torch.mm(A, B) # don't work, cuz B is not a tensor

tensor([[0.8937],
        [0.5191]], dtype=torch.float64)

In [ ]:
# ways to perform matrix multiplication
A = torch.randn(2,3)
B = torch.randn(3,1)
A @ B, A.mm(B), torch.mm(A,B), torch.matmul(A,B)

(tensor([[ 0.1354],
         [-1.6401]]), tensor([[ 0.1354],
         [-1.6401]]), tensor([[ 0.1354],
         [-1.6401]]), tensor([[ 0.1354],
         [-1.6401]]))

In [ ]:
%%timeit
A = torch.randn(2,3)
B = torch.randn(3,1)
A @ B

The slowest run took 99.86 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 5: 11.3 µs per loop


In [ ]:
%%timeit
A = torch.randn(2,3)
B = torch.randn(3,1)
A.mm(B) # a bit faster

The slowest run took 15.33 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 5: 10.6 µs per loop


In [ ]:
class SyntheticRegressionData_v2(d2l.DataModule):
    def __init__(self, w, b, noise = 0.01, num_train = 1000, num_val = 100, 
                 batch_size = 32):
        super().__init__()
        self.save_hyperparameters()
        n = num_train + num_val
        self.X = torch.randn(n, len(w))
        nosie = torch.randn(n,1)*noise
        self.y = torch.matmul(self.X, w.reshape((-1,1))) + b + noise

In [ ]:
class SyntheticRegressionData(d2l.DataModule):  #@save
    def __init__(self, w, b, noise=0.01, num_train=1000, num_val=1000,
                 batch_size=32):
        super().__init__()
        self.save_hyperparameters()
        n = num_train + num_val
        self.X = torch.randn(n, len(w))
        noise = torch.randn(n, 1) * noise
        self.y = torch.matmul(self.X, w.reshape((-1, 1))) + b + noise

Below, we set the true parameters to $\mathbf{w} = [2, -3.4]^\top$ and $b = 4.2$.
Later, we can check our estimated parameters against these *ground truth* values.


In [ ]:
data = SyntheticRegressionData(w=torch.tensor([2, -3.4]), b=4.2)

[**Each row in `features` consists of a vector in $\mathbb{R}^2$ and each row in `labels` is a scalar.**] Let's have a look at the first entry.


In [ ]:
data.X.shape,data.y.shape

(torch.Size([1100, 2]), torch.Size([1100, 1]))

In [ ]:
print('features:', data.X[0],'\nlabel:', data.y[0])

features: tensor([-0.4327, -0.0064]) 
label: tensor([3.3665])


## Reading the Dataset

Training machine learning models often requires multiple passes over a dataset, 
grabbing one minibatch of examples at a time. 
This data is then used to update the model. 
To illustrate how this works, we 
[**implement the `get_dataloader` function,**] 
registering it as a method in the `SyntheticRegressionData` class via `add_to_class` (introduced in :numref:`oo-design-utilities`).
It (**takes a batch size, a matrix of features,
and a vector of labels, and generates minibatches of size `batch_size`.**)
As such, each minibatch consists of a tuple of features and labels. 
Note that we need to be mindful of whether we're in training or validation mode: 
in the former, we will want to read the data in random order, 
whereas for the latter, being able to read data in a pre-defined order 
may be important for debugging purposes.


In [ ]:
indices = list(range(0, 20))
random.shuffle(indices)
indices

[13, 14, 2, 7, 6, 0, 11, 16, 15, 17, 12, 8, 1, 19, 10, 9, 5, 18, 3, 4]

In [ ]:
for i in range(1,20,3):
    print(i)

1
4
7
10
13
16
19


In [ ]:
indices = torch.tensor(indices)

In [ ]:
torch.randn(50,20)[indices[0], indices[0]] # just like indexing ndarray

tensor(0.2187)

In [ ]:
@d2l.add_to_class(SyntheticRegressionData) # add a method to previously defined class
def get_dataloader(self, train):
    if train:
        indices = list(range(0, self.num_train))
        # The examples are read in random order, shuffle inplace
        random.shuffle(indices)
    else:
        indices = list(range(self.num_train, self.num_train+self.num_val))
    for i in range(0, len(indices), self.batch_size):
        batch_indices = torch.tensor(indices[i: i+self.batch_size])
        yield self.X[batch_indices], self.y[batch_indices] # return a generator, how train module cooperater with this generator?

To build some intuition, let's inspect the first minibatch of
data. Each minibatch of features provides us with both its size and the dimensionality of input features.
Likewise, our minibatch of labels will have a matching shape given by `batch_size`.


In [ ]:
X, y = next(iter(data.train_dataloader()))
print('X shape:', X.shape, '\ny shape:', y.shape)

X shape: torch.Size([32, 2]) 
y shape: torch.Size([32, 1])


While seemingly innocuous, the invocation 
of `iter(data.train_dataloader())` 
illustrates the power of Python's object-oriented design. 
Note that we added a method to the `SyntheticRegressionData` class
*after* creating the `data` object. 
Nonetheless, the object benefits from 
the *ex post facto* addition of functionality to the class.

Throughout the iteration we obtain distinct minibatches
until the entire dataset has been exhausted (try this).
While the iteration implemented above is good for didactic purposes,
it is inefficient in ways that might get us in trouble on real problems.
For example, it requires that we load all the data in memory
and that we perform lots of random memory access.
The built-in iterators implemented in a deep learning framework
are considerably more efficient and they can deal
with sources such as data stored in files, 
data received via a stream, 
and data generated or processed on the fly. 
Next let's try to implement the same function using built-in iterators.

## Concise Implementation of the Data Loader

Rather than writing our own iterator,
we can [**call the existing API in a framework to load data.**]
As before, we need a dataset with features `X` and labels `y`. 
Beyond that, we set `batch_size` in the built-in data loader 
and let it take care of shuffling examples  efficiently.


In [ ]:
@d2l.add_to_class(d2l.DataModule)  #@save
def get_tensorloader(self, tensors, train, indices=slice(0, None)):
    tensors = tuple(a[indices] for a in tensors) # a equals to X or y. Use tuple to concatenate them again
dataset = torch.utils.data.TensorDataset(*tensors) # unpack X and y
    return torch.utils.data.DataLoader(dataset, self.batch_size,
                                       shuffle=train)
@d2l.add_to_class(SyntheticRegressionData)  #@save
def get_dataloader(self, train):
    i = slice(0, self.num_train) if train else slice(self.num_train, None)
    return self.get_tensorloader((self.X, self.y), train, i)

The new data loader behaves just as the previous one, except that it is more efficient and has some added functionality.


In [ ]:
X, y = next(iter(data.train_dataloader()))
print('X shape:', X.shape, '\ny shape:', y.shape)

X shape: torch.Size([32, 2]) 
y shape: torch.Size([32, 1])


For instance, the data loader provided by the framework API 
supports the built-in `__len__` method, 
so we can query its length, 
i.e., the number of batches.


In [ ]:
len(data.train_dataloader())

32

In [ ]:
data.get_tensorloader((data.X, data.y), 'train', slice(0, data.num_train))

In [ ]:
# slice(start, limit, step). slice(0, None) means ::, which takes all of the data!
slice(0,None),slice(0,20)
torch.randn(20,30)[slice(0,2)].shape,torch.randn(20,30)[slice(0,None, None)].shape

(torch.Size([2, 30]), torch.Size([20, 30]))

## Summary

Data loaders are a convenient way of abstracting out 
the process of loading and manipulating data. 
This way the same machine learning *algorithm* 
is capable of processing many different types and sources of data 
without the need for modification. 
One of the nice things about data loaders 
is that they can be composed. 
For instance, we might be loading images 
and then have a post-processing filter 
that crops them or modifies them otherwise. 
As such, data loaders can be used 
to describe an entire data processing pipeline. 

As for the model itself, the two-dimensional linear model 
is about as simple a model as we might encounter. 
It lets us test out the accuracy of regression models 
without worry about having insufficient amounts of data 
or an underdetermined system of equations. 
We will put this to good use in the next section.  


## Exercises

1. What will happen if the number of examples cannot be divided by the batch size. How to change this behavior by specifying a different argument by using framework's API?
1. What if we want to generate a huge dataset, where both the size of the parameter vector `w` and the number of examples `num_examples` are large? 
    1. What happens if we cannot hold all data in memory?
    1. How would you shuffle the data if data is held on disk? Your task is to design an *efficient* algorithm that does not require too many random reads or writes. Hint: [pseudorandom permutation generators](https://en.wikipedia.org/wiki/Pseudorandom_permutation) allow you to design a reshuffle without the need to store the permutation table explicitly :cite:`Naor.Reingold.1999`. 
1. Implement a data generator that produces new data on the fly, every time the iterator is called. 
1. How would you design a random data generator that generates *the same* data each time it's called?


In [ ]:
# Q1: It will return all data remaining
[1,2,3,4][3:6]

[4]

Q1:
1. It will return all data remaing

Q2: 
1. We need to read particular part of file from the disk
2. Although X(original data) is huge, we define a small data K, and use F(X,k) -> X to find a pseudorandom permutaion

Q3:
1. TBD

Q4:
1. Manually set seed with `torch.manual_seed()`. `torch.manual_seed()` set seed for CPU，`torch.cuda.manual_seed()` set seed for GPU，`torch.cuda.manual_seed_all()` set seed for all GPUs，`random.seed()` set seed for random module

In [ ]:
torch.manual_seed(0)
torch.randn(3,2)

tensor([[ 1.5410, -0.2934],
        [-2.1788,  0.5684],
        [-1.0845, -1.3986]])

In [ ]:
# import torch
 
torch.manual_seed(0)
print(torch.rand(1, 2)) # rand
 
torch.manual_seed(0)
print(torch.randn(1, 2)) # randn

tensor([[0.4963, 0.7682]])
tensor([[ 1.5410, -0.2934]])


[Discussions](https://discuss.d2l.ai/t/6663)
